# Volume Dependent Pressure Monte Carlo


## Some Background

We want to use concentration fluctuations in APT data to extract the free energy of mixing and the critical temperature of demixing in simple alloys.  

The theoretical model for this is cut the reconstruction volume into small spheres with constant number of particles and assume that the rest of the sample is acting like a bath for this particular sphere.  

The sphere may exchange
- heat,
- volume,
- and particle types (keeping the total number constant)

with its surroundings, so we expect a $N,\Delta\mu,p,T$ coupling.

We tried to simulate this by doing NVT-MC (particle swaps, but not flips + particle displacements) on a large sample and tracing the concentration fluctution in a small sub sample.  The concentration fluctuation when coupled with an ideal bath should behave as
$$
\sigma^2_c = \frac{k_B T}{N\partial^2_c f(c)}
$$

Problem: We see finite-size effects on $\partial^2_c f(c)$ even with large sub sample sizes (~1000 particles in >100000 particle box) and we also see them when evaluating experimental data.

## Distribution

I derived the distribution function of the local concentration in a larger NVT sample as
$$
\begin{align}
       p(c^\mathrm{s}|c, N^\mathrm{s}) &= \int \mathrm{d}\,V^\mathrm{s}
            \frac{Z^\mathrm{b}Z^\mathrm{s}}{Z}
            \exp{-\beta U_i(c^\mathrm{s}, c^\mathrm{b})} \\
        &\times \exp{-\beta \left[
                F_s(c^\mathrm{s}, N^\mathrm{s}, V^\mathrm{s})
                    - N^\mathrm{s} \Delta\mu c^\mathrm{s}
                    - p_\mathrm{inst} V^\mathrm{s}
                \right]} \\
        &\times \exp{-\beta \left[
                F_b(c, N^\mathrm{b}, V^\mathrm{b})
                    - N^\mathrm{s} \Delta\mu c
                    - p_\mathrm{inst} {\hat V}^\mathrm{s}
                \right]}.
\end{align}
$$
where `s` denotes system quantities and `b` bath quantities and
$$
p_\mathrm{inst}(V^\mathrm{b}) = p + \frac{1}{2}B \frac{\hat V^\mathrm{b} - V^\mathrm{b}}{V^\mathrm{b}}
$$
where $\hat V^\mathrm{b}$ is the most likely volume of the bath, $p = -\partial_V F^\mathrm{b}$ and $B$ is the bulk modulus of the bath.

This corresponds to a distribution according to a "modified" Gibbs Free Energy
$$
G^{\mathrm{s} \prime} = G^\mathrm{s} + B \frac{\Delta V^{\mathrm{s} 2}}{\hat V^{\mathrm{s}}}
$$

## Deviation from $N, \Delta\mu, p, T$

- the interaction term between system and bath $U^i$
- the "instantaneous" pressure

In alloys with a large system mismatch (here, Cu/Ag) we would expect a significant correlation between concentration and volume fluctuations.

To see if this pressure-dependent contribution can make a difference, I wanted to run simulations on just the small sub system where the pressure instantly reacts to the system volume changes.

For now, I only want to make sure I can control the pressure and not worry about the effect on the concentration fluctuations.

# (Attempt at an) Implementation

[Mishin](https://iopscience.iop.org/article/10.1088/0965-0393/22/4/045001/pdf) already gave an implementation for a finite-size reservoir for concentration fluctuations, so I wanted to see, if I can use a similar approach. 

Instead of running normal $N,p,T$ MC, adjust the pressure every step according to this scheme
1. set target volume $V_0$
2. pick a feedback constant $\alpha$
3. change the pressure according to $p^\prime = p + \alpha \frac{\Delta V}{V_0}$
4. run MC with $p^\prime$ for $n$ steps, goto 3.

## Problem: What is the coupling ($B$) induced by a feedback $\alpha$?

Similar to the concentration fluctuation above we can write for the volume fluctuation

$$
\sigma^2_V = \frac{k_B T}{\partial^2_V G^\prime} = \frac{k_B T}{\partial^2_V G + B \left(\frac{\Delta V}{V}\right)^2}
$$

Hope: $B = f(\alpha)$, i.e. plot $\sigma^2_V$ vs. $\alpha$ to find a relation.

# Advertising Intermezzo

[Sebastian Eich](https://www.imw.uni-stuttgart.de/team/Eich-00003/) and I develop an MD/MC code at University of Stuttgart.   Forked from a [code](https://en.wikipedia.org/wiki/XMD) from [Jon Rifkin](http://xmd.sourceforge.net/) at UConn. (Created just a few years before I was born. ;))

Designed for interactive use, has flexible python wrapper.

Can do simple MD, but we use it mostly for MC: NVT, NpT, semi-grand-canonical with efficient and parallel energy evaluations and thermodynamic intergration (from quasi-harmonic reference).

The python wrapper is similar to pyiron interactive jobs, but much faster and flexible because there is no interprocess communication.

Single-Node performance is similar to Lammps, but we only do shared memory.  Where we try to differ from lammps is simplicity and extensibility, one of my current projects is implementing flexible MC ensembles.

Last year I showed some Grand-canonical MC, which was also implemented in XMD.

## Code Sample of a Script 

In [ ]:
x.scale( (V0 / x.properties.bcur.prod() / 1e24)**(1/3) )
x.write('particle', file='pressure.xyz')

x.mc(x.properties.np * 10)

fname = f'results_s={args.steps}_v={args.volume_step}_a={args.feedback}.txt'
with open(fname, 'w') as f:
    f.write('# Volume [A^3], pressure [bar], Energy [eV]')
M = 1000
for _ in range(args.cycles//M + 1):
    
    Vs = np.empty(M)
    ps = np.empty(M)
    Es = np.empty(M)
    
    x.write('particle', file='+pressure.xyz')
    
    for i in range(M):
        # update feedback
        V = x.properties.bcur.prod() * 1e24 # funny unit conversions
        p += args.feedback * (V - V0) / V0

        x.reads(f"MCPARAMS PRESSURE {p}")
        x.mc(args.steps)

        # collect results
        print(V0, V, p)
        Vs[i] = V
        ps[i] = p
        Es[i] = x.properties.epot * 624e9

    with open(fname, 'ab') as f:
        np.savetxt(f, np.transpose([Vs, ps, Es]))

## Results

### Feedback applied every 5 MC steps

<img src="./pressure_s5_pressure.png">

<img src="./pressure_s5_volume.png">

### Feedback applied every 1 MC steps

<img src="./pressure_s1_pressure.png">

<img src="./pressure_s1_volume.png">

### Feedback applied every 50 MC steps

<img src="./pressure_s50_pressure.png">

<img src="./pressure_s50_volume.png">

# Does this break Markov-Chain Properties?

In the meeting we discussed whether this can break the reversibility of the Markov-Chain and instead talked about whether it might be better to set the instantaneaous pressure from a a-priori known $p(V)$ curve during the simulation.

Assuming the bulk modulus of the bath is constant this pressure would be (in the modified scheme)
$$
p(V) = p_0 - B \Delta V
$$
where $p_0$ is the pressure that stabilizes the system at the target volume $V_0$ (modulo prefactors, signs, etc.).

Since this is a linear dependency on $\Delta V$ this is in fact equivalent to the feedback approach.  Consider e.g. that the system starts the simulation at $p_0$, $V_0$ and the next step is a volume rescaling step to the volume $V^\prime$.  If it is accepted the new volume is $V^\prime$.  In the feedback scheme the new pressure and volume would be $p_0 + \alpha \frac{V_0 - V^\prime}{V_0}$, $V_0$ where as in the modified scheme it would be $p_0 - B (V_0 - V^\prime)$, $V^\prime$.  After accepting consider a possible step back to $V_0$, both schemes would update the pressure to $p_0$.  Identifying $\alpha = -B V_0$, shows that both schemes would update the pressure identically and hence would also follow the same acceptance probabilities.

If $p(V)$ is not linear, but the feedback is small this argument still holds as we may locally approximate $p(V)$ linearly.  The advantage of the feedback scheme is then that know a-priori knowledge of $p_0$ or $p(V)$ is needed.  As pointed out in the discussion picking a feedback value that is too large voids this argument.

# GCMC Sample Script

for a single GCMC step

In [ ]:
E0 = x.props.epot

# always consume random int for repro
add_atom = np.random.randint(0,2) and nvac > 0
if add_atom:
    q = find_candidate_insertion(x)
    add_particle(x, 10, x.calc.ATOMIC_MASS_1/6.022e23, q)
    x.write('particle', file = '+back.xyz')
    x.props.type[-1] = 0
else:
    # make sure we only select type 0
    I = index = np.random.choice(np.where(x.props.type==0)[0])
    t = x.props.type[I] + 1
    q = x.props.cur[I].copy()
    v = x.props.v[I].copy()
    f = x.props.f[I].copy()
    m = x.props.mass[I]

    x.select(index = I + 1)
    x.remove()

x.write('energy')
E = x.props.epot

### SNIP: some numerical fiddeling
...
### SNIP

dc = cvac - (nvac1 + nvac0)/2
µ += 1e-5 * EV2ERG * dc * abs(dc)
with open('mun.txt', 'a') as io:
    io.write(f"{µ} {nvac}\n")
if a: # step accepted
    E0 = E
    nvac += Δv
    nvac1 = nvac0
    nvac0 = nvac
else:
    if add_atom:
        x.write('particle', file = '+back.xyz')
        x.select(index=x.props.np - 1)
        x.remove()
        x.write('particle', file = '+back.xyz')
    else:
        add_particle(x, t, m, q, v, f)

x.cmd(100)
# x.mc(100)
x.write('particle', file = '+gcmc.xyz')
